<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_GaussianNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ガウス単純ベイズ識別（Gaussian naive Bayes classification）

----

氏名：

学生番号：

----
基本課題（必須）

    1. 「★scikit-learn を用いたガウス単純ベイズ識別」まで実行すると，何を表す数値と図が出力されますか．
       また，表示される数値と図から何が言えますか．特に，Example 2について記述してください．

（ここに回答を書いてください）



    2. 「★事前確率の計算」と「★標本平均と標本分散の計算」を完成させてください．
       また，その実行結果が，「★scikit-learn を用いたガウス単純ベイズ識別」で表示される
       "Prior probs"，"Mean"，"Variance" と一致することを確認してください．

（回答は「★事前確率の計算」と「★標本平均と標本分散の計算」に書いてください）



    3. 「★scikit-learn を用いたガウス単純ベイズ識別」または「★確率密度の比較」で表示される決定境界は，
       どのような曲線があり得るでしょうか．色々な設定でExampleを観察して確認してください．

（ここに回答を書いてください）  



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）



----

In [ ]:
#@title 2次元に2クラスを描く `plot2cls` を定義します（理解不要）．
!wget -q -N https://gist.githubusercontent.com/tsakai-g/1920bc185be0c497579968e00a9b3674/raw/plot2cls.py
%run plot2cls.py

### 実験用のデータ（2次元，2クラス）を生成します．
- Example 2～5 からひとつ選んで実行してください．

In [ ]:
# Example 2: draw npos and nneg points from the Gaussian distribution for each class
npos = 100
nneg = 500
np.random.seed(321)
X_all = np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]
# [1,1,...,1,-1,-1,...,-1]
y_all = np.array([1] * npos + [-1] * nneg)

In [ ]:
# Example 3: create moons using sklearn
from sklearn.datasets import make_moons
X_all, y_all = make_moons(n_samples=300, noise=0.3, random_state=0)
y_all[y_all==0] = -1

In [ ]:
# Example 4: create circles using sklearn
from sklearn.datasets import make_circles
X_all, y_all = make_circles(n_samples=300, noise=0.2, random_state=0, factor=0.3)
y_all[y_all==0] = -1

In [ ]:
# Example 5: combined dataset
from sklearn.datasets import make_moons
X_all, y_all = make_moons(n_samples=100, noise=0.2, random_state=0)
y_all[y_all==0] = -1
npos, nneg = 30, 300
np.random.seed(321)
X_all = np.vstack((2*X_all+np.array([4,-2]), np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]))
# [1,1,...,1,-1,-1,...,-1]
y_all = np.hstack((y_all, np.array([1] * npos + [-1] * nneg)))

### 訓練データ集合 `(X,y)` と検証データ集合 `(X_val,y_val)` に分けます．

In [ ]:
from sklearn.model_selection import train_test_split

X, X_val, y, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=0)
print("#train&val, #test =", X.shape[0], ",", X_val.shape[0])

In [ ]:
#@title データを表示します．
print("#X_all =", X_all.shape[0])
plot2cls(X, y, X_val=X_val, y_val=y_val).plot_data()

### ★scikit-learn を用いたガウス単純ベイズ識別

[sklearn.naive_bayes.GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)を使用した結果を出力します．

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X,y)

# クラス数（number of classes）
print("# of classes: ", len(model.class_count_))

# 各クラスの事前確率（prior probability of each class）
print("Prior probs: ", model.class_prior_)

# 各クラスの各特徴の標本平均と標本分散（mean and variance of each feature per class）
print("Mean: ", model.theta_.tolist())
print("Variance: ", model.var_.tolist())

print("Accuracy on training data: ", model.score(X, y))
print("Accuracy on validation data: ", model.score(X_val, y_val))

dec = lambda X: np.diff(model.predict_log_proba(X)).ravel()
plot2cls(X, y, dec, X_val, y_val).plot_clf()

In [ ]:
#@title 混同行列（行：正解，列：予測）
from sklearn import metrics
cm = metrics.confusion_matrix(y_val, model.predict(X_val))
print(cm)

from sklearn.metrics import classification_report
print(classification_report(y_val, model.predict(X_val)))

## ガウス単純ベイズ識別の計算を確認しましょう．

条件付き独立を仮定した生成確率密度 $P(\boldsymbol{x}|Y)=P(x_1|Y)P(x_2|Y)$,

各クラスの各特徴の生成確率密度 $P(x_i|Y) = \frac{1}{\sqrt{2\pi}\sigma_{i|Y}}\exp\left\{-\frac{(x_i-\mu_{i|Y})^2}{2\sigma_{i|Y}^2}\right\}$ （ただし，$Y\in\{+,-\}$, $i\in\{1,2\}$）

- **★事前確率の計算**：各クラスのデータ数の割合P(Y)を求めます．
- **★標本平均と標本分散の計算**：特徴が独立に正規分布することを仮定して，各クラスの訓練データから各特徴の標本平均（sample mean）$\mu_{1|+}$, $\mu_{2|+}$, $\mu_{1|-}$, $\mu_{2|-}$と，標本分散（sample variance）$\sigma_{1|+}$, $\sigma_{2|+}$, $\sigma_{1|-}$, $\sigma_{2|-}$を求めます（統計的学習）．
- （$\hat{\boldsymbol{\mu}}_+=[\hat\mu_{1|+},\hat\mu_{2|+}]^\top$，${\hat{\boldsymbol{\mu}}}_-=[\hat\mu_{1|-},\hat\mu_{2|-}]^\top$，$\hat{\boldsymbol{\Sigma}}_+=\left[\begin{array}{cc}\hat\sigma_{1|+}^2 & 0\\ 0 & \hat\sigma_{2|+}^2\end{array}\right]$，$\hat{\boldsymbol{\Sigma}}_-=\left[\begin{array}{cc}\hat\sigma_{1|-}^2 & 0\\ 0 & \hat\sigma_{2|-}^2\end{array}\right]$）
- **★確率密度の比較**：各クラスの事後確率密度を比較します（統計的推論）．事後確率密度の代わりに同時確率密度 $P(\boldsymbol{x},Y) = P(\boldsymbol{x}|Y)P(Y)$ を計算します．

### ★事前確率の計算

- 2クラスのデータは `X_p = X[y>0]` と `X_n = X[y<=0]` です．その各行は2次元の特徴ベクトル x です．
- クラス$+$ (`y>0`) と$-$ (`y<=0`) のデータ数の割合を `Prior[0]` と`Prior[1]` とします．

In [ ]:
X_p = X[y>0]
X_n = X[y<=0]
print(X_p.shape, X_n.shape)

Prior = np.zeros(2)
Prior[0], Prior[1] = ''' 事前確率を計算してください '''

print("Prior =", Prior)

### ★標本平均と標本分散の計算

- 2クラスのデータは `X_p = X[y>0]` と `X_n = X[y<=0]` です．その各行は2次元の特徴ベクトルです．
- 標本平均ベクトルをそれぞれ `mu_p`，`mu_n` とします（`mu_p.shape` と `mu_n.shape` は `(2,)` です）．それぞれ，標本平均ベクトル$[\mu_{1|+},\mu_{2|+}]^\top$と$[\mu_{1|-},\mu_{2|-}]^\top$を表します．
- `X_p` と `X_n` の各特徴の標本分散をそれぞれ `s_p`，`s_n` とします．`s_p.shape` と `s_n.shape` は `(2,)` です．

In [ ]:
mu_p = ''' X_p から標本平均を計算してください '''
print("[mu1_p, mu2_p] = ", mu_p)

s_p = ''' X_p から標本分散を計算してください '''
print("[s11_p, s22_p] = ", s_p)

mu_n = ''' X_p から標本平均を計算してください '''
print("[mu1_n, mu2_n] = ", mu_n)

s_n = ''' X_n から標本分散を計算してください '''
print("[s11_n, s22_n] = ", s_n)


### ★確率密度の比較

- 計算した事前確率，標本平均，標本分散を用いて，データと共に正規分布と境界を図示します．
- 表示した領域内の多数の点で，各クラスの生成分布の確率密度を計算し（統計的推論），確率密度を色の濃さと等高線で表します．

In [ ]:
#@title 各クラスの2次元ガウス分布を図示する関数 `plot2d_GaussianNB` を定義します（理解不要）．
import numpy as np
# Independent bivariate normal joint density
def N2d(x, mu, variance):
    return np.exp(-0.5 * (x[:,0] - mu[0])**2 / variance[0])  *  np.exp(-0.5 * (x[:,1] - mu[1])**2 / variance[1])  /  (2. * np.pi * np.sqrt(variance[0]*variance[1]))
    #return np.exp(-0.5 * (x[:,0] - mu[0])**2 / variance[0])  *  np.exp(-0.5 * (x[:,1] - mu[1])**2 / variance[1])

from matplotlib import pyplot as plt
# Visualization of the estimated distributions
def plot2d_GaussianNB(X_train, y_train, pri, mus, vars, X_val=None, y_val=None, cmap=None, xlim=None, ylim=None, levels=None, linestyles=None, markers=None, colors=None):

    plt.figure(figsize=(8,8))
    ax = plt.axes()

    dlim = np.sqrt(vars).sum(axis=0)/2
    dxlim, dylim = dlim[0], dlim[1]

    if xlim is None:
        xlim = [X_train[:, 0].min() - dxlim, X_train[:, 0].max() + dxlim]
    if ylim is None:
        ylim = [X_train[:, 1].min() - dylim, X_train[:, 1].max() + dylim]

    xg, yg = np.arange(xlim[0], xlim[1], (xlim[1]-xlim[0])/300.), np.arange(ylim[0], ylim[1], (ylim[1]-ylim[0])/300.)
    xx, yy = np.meshgrid(xg, yg)

    if cmap is None:
        cmap = ['Reds', 'Blues', 'Greens', 'BuPu', 'RdPu', 'YlGn']
    if markers is None:
        markers = ['o', 's', '^', '*', '+', 'x']
    if colors is None:
        colors = ['b', 'r', 'g', 'c', 'm', 'y']
    ncmap, nm, nc = len(cmap), len(markers), len(colors)

    ncls = len(pri)
    vmax = 1.  / (2. * np.pi * np.sqrt(vars[:,0] * vars[:,1]))
    for k in range(ncls):
        #mean and variance of each feature per class
        mu = mus[k,:]
        variance = vars[k,:]
        Z = N2d(np.c_[xx.ravel(),yy.ravel()], mu, variance)

        # Put the result into a color plot
        #ax.pcolor(xx, yy, Z, cmap=cmap[k], alpha=0.5, edgecolors=None)
        Zm = np.ma.masked_array(Z, Z < 0.03*vmax[k])
        ax.pcolorfast(xg, yg, Zm.reshape(xx.shape), cmap=cmap[k%ncmap], alpha=0.5)
        if levels is not None:
            ax.contour(xx, yy, Z.reshape(xx.shape), levels=levels, colors='k', linestyles=linestyles, alpha=0.2)
        else:
            levels = np.arange(0, vmax[k], vmax[k]/8.)
            ax.contour(xx, yy, Z.reshape(xx.shape), levels=levels, colors='k', linestyles=linestyles, alpha=0.2)

    if ncls == 2:
        decision_function = lambda x: N2d(x, mus[1], vars[1])*pri[1] - N2d(x, mus[0], vars[0])*pri[0]
        Z = decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        ax.contour(xx, yy, Z.reshape(xx.shape), levels=[0.], colors='k', linestyles=['--'], alpha=1)

    # Plot also the training points
    y = np.unique(y_train)
    for k in range(ncls):
        ax.scatter(X_train[y_train==y[k], 0], X_train[y_train==y[k], 1], c=colors[k%nc],  marker=markers[k%nm], cmap=cmap[k%ncmap], edgecolors='k', label='Training data', alpha=1)
    # and val points if given
    if X_val is not None and y_val is not None:
        y = np.unique(y_train)
        for k in range(ncls):
            ax.scatter(X_val[y_val==y[k], 0], X_val[y_val==y[k], 1], c='k', marker=markers[k%nm], cmap=cmap[k], edgecolors='k', label='Val data', alpha=0.2)
        plt.legend(loc="upper right", fontsize=16, frameon=True, bbox_to_anchor=(0,1))

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    plt.axis('tight')
    plt.xlabel('x1', fontsize=16)
    plt.ylabel('x2', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.gca().set_aspect('equal')
    plt.tight_layout()
    #plt.savefig('rnd2d_ex1_GNB.png', transparent=True,dpi=300)

In [ ]:
plot2d_GaussianNB(X, y, Prior, np.c_[mu_p, mu_n].T, np.c_[s_p, s_n].T, X_val, y_val)

----
## 付録：2変量正規分布（Bivariate normal distribution）

${\rm N}(\mathbf{x};\mathbf{\mu},\mathbf{\Sigma})=\dfrac{1}{\sqrt{(2\pi)^m|\mathbf{\Sigma}|}}\,e^{-\frac{1}{2}(\mathbf{x}-\mathbf{\mu})^\top\Sigma^{-1}(\mathbf{x}-\mathbf{\mu})}$

$\mathbf{x}=\left[\matrix{x_1\\x_2}\right]$, $\mathbf{\mu}=\left[\matrix{\mu_1\\\mu_2}\right]$, $\mathbf{\Sigma}=\left[\matrix{s_{11}&s_{12}\\s_{21}&s_{22}}\right]$

In [ ]:
#@title  { run: "auto" }
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

mu1 = 0 #@param {type:"slider", min:-5, max:5, step:0.1}
mu2 = 0 #@param {type:"slider", min:-5, max:5, step:0.1}
s11 = 1 #@param {type:"slider", min:0.00, max:4, step:0.1}
s12 = 0 #@param {type:"slider", min:-4, max:4, step:0.1}
s22 = 1 #@param {type:"slider", min:0.00, max:4, step:0.1}

if s12*s12 >= s11*s22:
    print("s12*s12 must be smaller than s11*s22")
    s12 = np.sign(s12)*(s11*s22-1e-8)

xmin, xmax = -5,5
x1, x2 = np.mgrid[xmin:xmax:.01, xmin:xmax:.01]
x = np.dstack((x1, x2))

prob = stats.multivariate_normal([mu1, mu2], [[s11, s12], [s12, s22]])

plt.figure()
ax = plt.axes()

ax.contourf(x1, x2, prob.pdf(x), alpha=0.9, cmap=plt.cm.Greens)
label = '$\\mu=$['+"{:.2f}".format(mu1)+', '+"{:.2f}".format(mu2)+']\n$\\Sigma=$[['+"{:.2f}".format(s11)+', '+"{:.2f}".format(s12)+']\n        ['+"{:.2f}".format(s12)+', '+"{:.2f}".format(s22)+']]'
plt.text(1, 3, label, fontsize=16)
plt.xlim(xmin, xmax)
plt.ylim(xmin, xmax)
plt.grid(True, ls='--', lw=0.5, color='k')
plt.xlabel("$x_1$", fontsize=20)
plt.ylabel("$x_2$", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.set_aspect('equal')
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
plt.tight_layout()

お疲れさまでした．

--------